# <u>*Lasso Regression (Least Absolute Shrinkage and Selection Operator.)*<u>

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler # Import StandardScaler
from sklearn.model_selection import KFold
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, median_absolute_error

from IPython.display import Markdown # Import Markdown
from sklearn.metrics import r2_score


# DATA PROCESSING

In [2]:
df = pd.read_csv('AAPL.csv', delimiter=';')
df['Date'] = pd.to_datetime(df['Date'], format='%d.%m.%Y', errors='coerce')
df.head()

,Date,PX_LAST,Returns,PX_OPEN,PX_HIGH,PX_LOW,VOLUME,VWAP,VIX,PX_ASK,...,VOLATILITY_90D,Mkt-RF,SMB,HML,RMW,CMA,RF,Price Earnings Ratio (P/E),BEst P/E Ratio,Price/Cash Flow
0,2010-01-04,7.643,0.000000,7.623,7.661,7.585,493728200,7.6314,20.04,7.655,...,25.459,1.69,0.79,1.12,-0.17,0.21,0.0,20.90,23.23,16.04
1,2010-01-05,7.656,0.001701,7.664,7.700,7.616,601904016,7.6657,19.35,7.656,...,25.408,0.31,-0.41,1.24,-0.19,0.19,0.0,20.94,23.27,16.06
2,2010-01-06,7.535,-0.015805,7.656,7.687,7.527,552158376,7.6068,19.16,7.534,...,25.607,0.13,-0.13,0.57,-0.05,0.20,0.0,20.60,22.42,15.81
3,2010-01-07,7.521,-0.001858,7.563,7.571,7.466,477078140,7.5124,19.06,7.519,...,25.515,0.40,0.25,0.98,-0.69,0.22,0.0,20.56,21.74,15.78
4,2010-01-08,7.571,0.006648,7.511,7.571,7.466,447876324,7.5324,18.13,7.571,...,25.289,0.33,0.32,0.01,0.22,-0.37,0.0,20.70,21.88,15.88


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3774 entries, 0 to 3773
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Date                        3774 non-null   datetime64[ns]
 1   PX_LAST                     3774 non-null   float64       
 2   Returns                     3774 non-null   float64       
 3   PX_OPEN                     3774 non-null   float64       
 4   PX_HIGH                     3774 non-null   float64       
 5   PX_LOW                      3774 non-null   float64       
 6   VOLUME                      3774 non-null   int64         
 7   VWAP                        3774 non-null   float64       
 8   VIX                         3774 non-null   float64       
 9   PX_ASK                      3774 non-null   float64       
 10  PX_BID                      3774 non-null   float64       
 11  SPREAD                      3774 non-null   float64     

In [4]:
# Check for missing data
print("Missing values per column:")
print(df.isnull().sum())

Missing values per column:
Date                          0
PX_LAST                       0
Returns                       0
PX_OPEN                       0
PX_HIGH                       0
PX_LOW                        0
VOLUME                        0
VWAP                          0
VIX                           0
PX_ASK                        0
PX_BID                        0
SPREAD                        0
HIST_CALL_IMP_VOL             0
VOLATILITY_30D                0
VOLATILITY_60D                0
VOLATILITY_90D                0
Mkt-RF                        0
SMB                           0
HML                           0
RMW                           0
CMA                           0
RF                            0
Price Earnings Ratio (P/E)    0
BEst P/E Ratio                0
Price/Cash Flow               0
dtype: int64


In [5]:
# drop date
df_features = df.drop(['Date'], axis=1)

# Define X and y
X = df_features.drop(columns=['Returns'])  # Features colums: All columns except 'Returns'
y = df_features['Returns']                 # Target column

# Convert to numpy arrays
# X = X.to_numpy()
# y = y.to_numpy()

In [6]:
np.random.seed(1)

# Step 1: Split into train (70%) and temp (30%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, random_state=2)

# Step 2: Split temp into validation (15%) and test (15%)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, random_state=2)

In [7]:
# Check shapes of the datasets
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)

print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (2641, 23)
y_train shape: (2641,)
X_val shape: (566, 23)
y_val shape: (566,)
X_test shape: (567, 23)
y_test shape: (567,)


In [8]:
# To scale we use StandardScaler() since it is commonly used in models that assume normally distributed inputs
# or perform linear operations,such as: MLR, Ridge and Lasso

# -Apply Scaling
scaler_X = StandardScaler()

# Fit only on training data
X_train_scaled = scaler_X.fit_transform(X_train)

# Transform validation and test sets
X_val_scaled = scaler_X.transform(X_val)

-----
## Lasso Regression Model Summary on Standardized Data ##
This model fit is used as a baseline benchmark, where all variables are included, even those that do not fully satisfy the model assumptions. The purpose is to establish a reference point against which subsequent models—refined by excluding problematic features—can be evaluated to assess improvements in performance and robustness.

In [9]:
lasso = Lasso()

In [10]:
# Set up a range of possible lambda values
param_grid = {
    'alpha' : [0.00000001, 0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
}

kf = KFold(n_splits=10, shuffle=True, random_state=42)

lasso_cv = GridSearchCV(lasso, param_grid, cv=kf, n_jobs=-1)

lasso_cv.fit(X_train_scaled, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.684e-01, tolerance: 8.215e-05
  model = cd_fast.enet_coordinate_descent(


GridSearchCV(cv=KFold(n_splits=10, random_state=42, shuffle=True),
             estimator=Lasso(), n_jobs=-1,
             param_grid={'alpha': [1e-08, 1e-06, 1e-05, 0.0001, 0.001, 0.01,
                                   0.1, 1.0, 10.0, 100.0]})

**Comment:** The optimal regularization parameter selected via cross-validation is **$\lambda = 1 \times 10^{-6}$**,  
which is extremely small and therefore indicates almost no shrinkage of the coefficients.  
Such a low value of $\lambda$ implies that the penalty term has negligible influence on the estimation process,  
and the Lasso model behaves very similarly to ordinary least squares (OLS), providing virtually no regularization.

In [11]:
# General Summary
def lasso_model_info(model, X, y):
    n, k = X.shape
    y_pred = model.predict(X)
    residuals = y - y_pred
    SSR = np.sum((y_pred - np.mean(y))**2)
    RSS = np.sum((y - y_pred)**2)
    TSS = SSR + RSS
    R2 = SSR / TSS
    R2_adj = 1 - (1 - R2) * (n - 1) / (n - k - 1)

    mse = RSS / (n - k)
    log_likelihood = -0.5 * n * (np.log(2 * np.pi) + np.log(mse) + 1)
    aic = -2 * log_likelihood + 2 * k
    bic = -2 * log_likelihood + np.log(n) * k

    dw = np.sum(np.diff(residuals)**2) / RSS

    # Jarque-Bera test for normality of residuals
    skew = stats.skew(residuals)
    kurt = stats.kurtosis(residuals, fisher=False)
    jb_stat = n / 6 * (skew**2 + 0.25 * (kurt - 3)**2)
    jb_p = 1 - stats.chi2.cdf(jb_stat, df=2)

    # Condition number of design matrix
    X_design = np.hstack([np.ones((n, 1)), X]) if model.fit_intercept else X
    cond_no = np.linalg.cond(X_design)

    return {
        '============GENERAL SUMMARY OF MODEL ============': '',
        'R-squared': R2,
        'Adjusted R-squared': R2_adj,
        'Log-Likelihood': log_likelihood,
        'AIC': aic,
        'BIC': bic,
        'Durbin-Watson': dw,
        'Skew': skew,
        'Kurtosis': kurt,
        'Jarque-Bera': jb_stat,
        'Prob(JB)': jb_p,
        'Condition Number': cond_no,
        '============END OF GENERAL SUMMARY ============': ''

    }

def lasso_summary(model, X, y, feature_names=None):

    n, k = X.shape
    y_pred = model.predict(X)
    residuals = y - y_pred
    RSS = np.sum(residuals ** 2)
    MSE = RSS / (n - k)

    # If intercept is used, add intercept column to X
    if model.fit_intercept:
        X = np.hstack([np.ones((X.shape[0], 1)), X])
        coef = np.concatenate([[model.intercept_], model.coef_])
        k += 1
    else:
        coef = model.coef_

    # Variance-covariance matrix
    alpha = model.alpha_ if hasattr(model, 'alpha_') else model.alpha
    XTX_inv = np.linalg.inv(X.T @ X + alpha * np.identity(k))
    var_beta = MSE * XTX_inv
    std_err = np.sqrt(np.diag(var_beta))

    # t-statistics and p-values
    t_stats = coef / std_err
    p_values = 2 * (1 - stats.t.cdf(np.abs(t_stats), df=n - k))

    # 95% confidence intervals
    ci_low = coef - 1.96 * std_err
    ci_high = coef + 1.96 * std_err

    if feature_names is None:
        feature_names = [f'x{i+1}' for i in range(k)]
    if model.fit_intercept:
        feature_names = ['const'] + feature_names

    summary = pd.DataFrame({
        'coef': coef,
        'std err': std_err,
        't': t_stats,
        'P>|t|': p_values,
        # '[0.025': ci_low,
        # '0.975]': ci_high
    }, index=feature_names)

    return summary


In [12]:
# Get the best Lasso model
lasso_optimal = lasso_cv.best_estimator_

# General Summary
info = lasso_model_info(lasso_optimal, X_train_scaled, y_train)
for k, v in info.items():
    if isinstance(v, str):
        print(k)
    else:
        print(f"{k}: {v:.4f}")

============GENERAL SUMMARY OF MODEL ============
R-squared: 0.5685
Adjusted R-squared: 0.5647
Log-Likelihood: 8038.7714
AIC: -16031.5429
BIC: -15896.3279
Durbin-Watson: 1.9786
Skew: -0.2394
Kurtosis: 11.6042
Jarque-Bera: 8171.9399
Prob(JB): 0.0000
Condition Number: 26871339731831580.0000
============END OF GENERAL SUMMARY ============


*Comment:* Multicollinearity is a serious issue (very high condition number).

In [13]:
def lasso_summary(model, X, y, feature_names=None):
    n, k = X.shape
    y_pred = model.predict(X)
    residuals = y - y_pred
    RSS = np.sum(residuals ** 2)
    MSE = RSS / (n - k)

    # If intercept is used, add intercept column to X
    if model.fit_intercept:
        X_design = np.hstack([np.ones((X.shape[0], 1)), X])
        coef = np.concatenate([[model.intercept_], model.coef_])
        k += 1
    else:
        X_design = X
        coef = model.coef_

    # Variance-covariance matrix (approximate, as Lasso doesn't have exact SEs)
    alpha = model.alpha_ if hasattr(model, 'alpha_') else model.alpha
    XTX_inv = np.linalg.inv(X_design.T @ X_design + alpha * np.identity(k))
    var_beta = MSE * XTX_inv
    std_err = np.sqrt(np.diag(var_beta))

    # t-statistics and p-values
    t_stats = coef / std_err
    p_values = 2 * (1 - stats.t.cdf(np.abs(t_stats), df=n - k))

    # Create feature name list
    if feature_names is None:
        feature_names = [f'x{i+1}' for i in range(k)]
    if model.fit_intercept:
        feature_names = ['const'] + feature_names

    # Build DataFrame
    summary = pd.DataFrame({
        'coef': coef,
        'std err': std_err,
        't': t_stats,
        'P>|t|': p_values
    }, index=feature_names)

    return summary


In [14]:
# Get the best model from GridSearchCV
lasso_optimal = lasso_cv.best_estimator_

# Generate the detailed coefficient summary
summary_df = lasso_summary(
    lasso_optimal,
    X_train_scaled,
    y_train,
    feature_names=X_train.columns.tolist()
)

# Display the summary
print(summary_df)


                                    coef   std err             t         P>|t|
const                       1.066778e-03  0.000224  4.754629e+00  2.095841e-06
PX_LAST                    -3.641463e-04  0.268631 -1.355563e-03  9.989185e-01
PX_OPEN                    -6.434458e-02  0.027815 -2.313326e+00  2.078186e-02
PX_HIGH                     3.702873e-02  0.048027  7.709964e-01  4.407787e-01
PX_LOW                     -2.767994e-03  0.045527 -6.079860e-02  9.515242e-01
VOLUME                     -2.754267e-05  0.000305 -9.021002e-02  9.281272e-01
VWAP                        1.118066e-02  0.085268  1.311236e-01  8.956876e-01
VIX                         8.593198e-04  0.000399  2.153847e+00  3.134321e-02
PX_ASK                      1.717807e-02  8.154863  2.106482e-03  9.983194e-01
PX_BID                      5.478024e-07  8.153615  6.718522e-08  9.999999e-01
SPREAD                     -2.642422e-04  0.002561 -1.031877e-01  9.178219e-01
HIST_CALL_IMP_VOL          -2.420371e-03  0.000427 -

*Comment:* Not significant ($p > 0.1$) – PX\_LAST, PX\_HIGH, PX\_LOW, VOLUME, VWAP, PX\_ASK, PX\_BID, SPREAD, VOLATILITY\_30D, VOLATILITY\_60D, VOLATILITY\_90D, CMA, P/E ratio, Price/Cash Flow.

Their coefficients should be interpreted cautiously as they do not provide strong evidence of explanatory power.

-----
## Baseline Prediction ##
In this section, we establish a baseline on the validation set. The objective is to benchmark the model’s predictive performance prior to addressing assumption violations, providing a reference point to evaluate improvements achieved through subsequent model refinement.

In [15]:
# Optimal lambda automatically selected and stored in 'ridge_cv_model.alpha_'
y_pred = lasso_cv.predict(X_val_scaled)

In [16]:
# --- Directional Accuracy ---
def directional_accuracy(y_true, y_pred):
    # Shift actual and predicted values to compute the direction
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Calculate the direction of actual and predicted change
    actual_direction = np.sign(y_true[1:] - y_true[:-1])
    predicted_direction = np.sign(y_pred[1:] - y_pred[:-1])

    # Compute directional accuracy
    return np.mean(actual_direction == predicted_direction)

# Evaluation metrics
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_val, y_pred)
mad = median_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)
direction_acc = directional_accuracy(y_val, y_pred)

# Print evaluation results
print(f"Mean Squared Error (MSE): {mse:.6f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.6f}")
print(f"Mean Absolute Error (MAE): {mae:.6f}")
print(f"Median Absolute Deviation (MAD): {mad:.6f}")
print(f"R-squared: {r2:.6f}")
print(f"Directional Accuracy: {direction_acc:.6f}")

Mean Squared Error (MSE): 0.000160
Root Mean Squared Error (RMSE): 0.012634
Mean Absolute Error (MAE): 0.008602
Median Absolute Deviation (MAD): 0.005943
R-squared: 0.484577
Directional Accuracy: 0.759292


-----
## Handling Multicollinearity ##
To address multicollinearity, we will identify highly correlated features ($r > 0.9$)
and remove them from the model.


### Correlation matrix ###

In [17]:
print(X_train.shape)
print(X_train_scaled.shape)
print(y_train.shape)

(2641, 23)
(2641, 23)
(2641,)


In [18]:
# Combine features and target into one DataFrame
train_data = pd.concat([X_train, y_train], axis=1)

# Calculate the correlation matrix
corr_matrix = train_data.corr()

# Unstack the correlation matrix
stacked_corr = corr_matrix.unstack()

# Remove self-correlations
filtered_corr = stacked_corr[stacked_corr != 1]

# Keep only pairs with correlation above 0.9
high_corr = filtered_corr[filtered_corr > 0.9]

# Drop duplicate pairs (e.g., (A, B) and (B, A))
high_corr = high_corr[~high_corr.index.duplicated(keep='first')]

# get highly correlated features
def get_highly_correlated_features(df, threshold=0.9):
    # Compute absolute correlation matrix
    corr_matrix = df.corr().abs()

    # Keep upper triangle of the matrix (excluding self-correlations)
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

    # Identify features to consider for removal
    features_to_remove = set()
    for column in upper.columns:
        high_corr_features = upper.index[upper[column] > threshold].tolist()
        if high_corr_features:
            features_to_remove.add(column)

    return sorted(features_to_remove)

# Example usage:
features_to_remove = get_highly_correlated_features(X_train, threshold=0.9)

print("Features to consider removing due to high correlation:")
print(features_to_remove)



Features to consider removing due to high correlation:
['BEst P/E Ratio', 'PX_ASK', 'PX_BID', 'PX_HIGH', 'PX_LOW', 'PX_OPEN', 'Price/Cash Flow', 'VWAP']


In [19]:
features_to_remove = [
    "BEst P/E Ratio",
    "PX_ASK",
    "PX_BID",
    "PX_HIGH",
    "PX_LOW",
    "PX_OPEN",
    "Price/Cash Flow",
    "VWAP"
]

X_train = X_train.drop(columns=features_to_remove)

In [20]:
# Check shapes of the datasets
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)

print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (2641, 15)
y_train shape: (2641,)
X_val shape: (566, 23)
y_val shape: (566,)
X_test shape: (567, 23)
y_test shape: (567,)


In [21]:
features_to_remove = [
    "BEst P/E Ratio",
    "PX_ASK",
    "PX_BID",
    "PX_HIGH",
    "PX_LOW",
    "PX_OPEN",
    "Price/Cash Flow",
    "VWAP"
]
X_val = X_val.drop(columns=features_to_remove)
X_test = X_test.drop(columns=features_to_remove)

In [22]:
print("X_val shape:", X_val.shape)
print("X_test shape:", X_test.shape)

X_val shape: (566, 15)
X_test shape: (567, 15)


In [23]:
X_train.head(1)

,PX_LAST,VOLUME,VIX,SPREAD,HIST_CALL_IMP_VOL,VOLATILITY_30D,VOLATILITY_60D,VOLATILITY_90D,Mkt-RF,SMB,HML,RMW,CMA,RF,Price Earnings Ratio (P/E)
1664,27.045,74641736,11.55,0.002,17.642,23.16,20.919,24.054,-0.06,0.14,-0.17,-0.14,-0.28,0.001,12.7


-----
## Relaxed Lasso ##
The Relaxed Lasso is a two-step extension of the Lasso method designed to reduce the bias introduced by strong regularization.  

- Step 1: Lasso is applied to perform variable selection by shrinking some coefficients exactly to zero.  
- Step 2: The model is refitted using only the selected predictors, but with weaker or no regularization, to obtain less biased coefficient estimates while retaining the sparsity achieved in the first stage.  

In [24]:
# Apply Scaling
scaler_X = StandardScaler()

# Fit only on training data
X_train_scaled = scaler_X.fit_transform(X_train)

# Transform validation and test sets
X_val_scaled = scaler_X.transform(X_val)

In [25]:
X_train_scaled.shape
X_val_scaled.shape

(566, 15)

**Comment:**  
For the Relaxed Lasso, I employed a grid of larger $\alpha$ values
($0.001, 0.01, 0.1, 1.0, 10.0, 100.0$) compared to the standard Lasso.  
The motivation is that stronger penalization in the first step encourages
more aggressive feature selection by shrinking additional coefficients to zero.  
In the second step, the model is refitted on the selected variables with reduced
or no regularization, thereby mitigating the bias introduced by the initial shrinkage.

In [26]:
lasso = Lasso()
# Set up a range of possible lambda values
param_grid = {
    'alpha' : [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
}

kf = KFold(n_splits=10, shuffle=True, random_state=42)

lasso_cv = GridSearchCV(lasso, param_grid, cv=kf, n_jobs=-1)

lasso_cv.fit(X_train_scaled, y_train)

GridSearchCV(cv=KFold(n_splits=10, random_state=42, shuffle=True),
             estimator=Lasso(), n_jobs=-1,
             param_grid={'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]})

**Comment:** Cross-validation selected only a very mild level of regularization, indicating that the data contains sufficient signal such that excluding too many variables would harm predictive performance.
The small value of **$\alpha$** suggests that strong penalization is unnecessary, as excessive shrinkage would remove relevant predictors.

In [27]:
# Get the best model from GridSearchCV
lasso_optimal = lasso_cv.best_estimator_

# Generate the detailed coefficient summary
summary_df = lasso_summary(
    lasso_optimal,
    X_train_scaled,
    y_train,
    feature_names=X_train.columns.tolist()
)

# Display the summary
print(summary_df)

                                coef   std err          t         P>|t|
const                       0.001067  0.000236   4.519738  6.465900e-06
PX_LAST                     0.000000  0.000692   0.000000  1.000000e+00
VOLUME                     -0.000000  0.000313  -0.000000  1.000000e+00
VIX                        -0.000000  0.000415  -0.000000  1.000000e+00
SPREAD                     -0.000000  0.000287  -0.000000  1.000000e+00
HIST_CALL_IMP_VOL          -0.000015  0.000433  -0.034534  9.724539e-01
VOLATILITY_30D             -0.000000  0.000455  -0.000000  1.000000e+00
VOLATILITY_60D             -0.000000  0.000668  -0.000000  1.000000e+00
VOLATILITY_90D             -0.000000  0.000538  -0.000000  1.000000e+00
Mkt-RF                      0.011629  0.000268  43.429946  0.000000e+00
SMB                        -0.000053  0.000269  -0.196229  8.444458e-01
HML                        -0.002722  0.000312  -8.716676  0.000000e+00
RMW                         0.001328  0.000271   4.905307  9.898

In [28]:
# Extract coefficients and feature names
coefficients = lasso_optimal.coef_
feature_names = X_train.columns.tolist()

# Zip together and filter out zero-coefficient features
zero_coef_features = [name for coef, name in zip(coefficients, feature_names) if coef == 0.0]

# Print the list of excluded features
print("Features with zero coefficients (excluded by Lasso):")
print(zero_coef_features)


Features with zero coefficients (excluded by Lasso):
['PX_LAST', 'VOLUME', 'VIX', 'SPREAD', 'VOLATILITY_30D', 'VOLATILITY_60D', 'VOLATILITY_90D', 'CMA', 'RF', 'Price Earnings Ratio (P/E)']


**Comment:**  The Lasso procedure set the coefficients of **VOLUME** and **VOLATILITY\_60D** to zero,  
indicating that these variables do not contribute explanatory power to the model.  
Consequently, they will be excluded from the final specification.

In [29]:
# Automatically get features with zero coefficients
features_to_remove = [name for coef, name in zip(coefficients, feature_names) if coef == 0.0]

# Remove them from all datasets
X_train = X_train.drop(columns=features_to_remove)
X_val = X_val.drop(columns=features_to_remove)
X_test = X_test.drop(columns=features_to_remove)

In [30]:
# Check shapes of the datasets
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)

print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (2641, 5)
y_train shape: (2641,)
X_val shape: (566, 5)
y_val shape: (566,)
X_test shape: (567, 5)
y_test shape: (567,)


In [31]:
# Apply Scaling
scaler_X = StandardScaler()

# Fit only on training data
X_train_scaled = scaler_X.fit_transform(X_train)

# Transform validation and test sets
X_val_scaled = scaler_X.transform(X_val)

In [32]:
print("X_train scaled shape:", X_train_scaled.shape)
print("X_val scaled shape:", X_val_scaled.shape)

X_train scaled shape: (2641, 5)
X_val scaled shape: (566, 5)


In [33]:
X_train.head(1)

,HIST_CALL_IMP_VOL,Mkt-RF,SMB,HML,RMW
1664,17.642,-0.06,0.14,-0.17,-0.14


In [34]:
lasso = Lasso()
# Set up a range of possible lambda values
param_grid = {
    'alpha' : [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
}

kf = KFold(n_splits=10, shuffle=True, random_state=42)

lasso_cv = GridSearchCV(lasso, param_grid, cv=kf, n_jobs=-1)

lasso_cv.fit(X_train_scaled, y_train)

GridSearchCV(cv=KFold(n_splits=10, random_state=42, shuffle=True),
             estimator=Lasso(), n_jobs=-1,
             param_grid={'alpha': [1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1, 1.0,
                                   10.0, 100.0]})

In [35]:
# Get the best Lasso model
lasso_optimal = lasso_cv.best_estimator_

# General Summary
info = lasso_model_info(lasso_optimal, X_train_scaled, y_train)
for k, v in info.items():
    if isinstance(v, str):
        print(k)
    else:
        print(f"{k}: {v:.4f}")

============GENERAL SUMMARY OF MODEL ============
R-squared: 0.5480
Adjusted R-squared: 0.5472
Log-Likelihood: 7962.5907
AIC: -15915.1814
BIC: -15885.7869
Durbin-Watson: 1.9588
Skew: -0.2232
Kurtosis: 10.9370
Jarque-Bera: 6954.1171
Prob(JB): 0.0000
Condition Number: 1.8241
============END OF GENERAL SUMMARY ============


In [36]:
# Get the best model from GridSearchCV
lasso_optimal = lasso_cv.best_estimator_

# Generate the detailed coefficient summary
summary_df = lasso_summary(
    lasso_optimal,
    X_train_scaled,
    y_train,
    feature_names=X_train.columns.tolist()
)

# Display the summary
print(summary_df)

                       coef   std err          t     P>|t|
const              0.001067  0.000231   4.619439  0.000004
HIST_CALL_IMP_VOL -0.000880  0.000233  -3.769545  0.000167
Mkt-RF             0.013115  0.000248  52.933833  0.000000
SMB               -0.000682  0.000262  -2.606795  0.009191
HML               -0.003931  0.000251 -15.673097  0.000000
RMW                0.002846  0.000264  10.789377  0.000000


-----
## Prediction on Validation Set ##

In [37]:
# Optimal lambda automatically selected and stored in 'ridge_cv_model.alpha_'
y_pred = lasso_cv.predict(X_val_scaled)

In [38]:
# Evaluation metrics
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_val, y_pred)
mad = median_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)
direction_acc = directional_accuracy(y_val, y_pred)

# Print evaluation results
print(f"Mean Squared Error (MSE): {mse:.6f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.6f}")
print(f"Mean Absolute Error (MAE): {mae:.6f}")
print(f"Median Absolute Deviation (MAD): {mad:.6f}")
print(f"R-squared: {r2:.6f}")
print(f"Directional Accuracy: {direction_acc:.6f}")

Mean Squared Error (MSE): 0.000172
Root Mean Squared Error (RMSE): 0.013121
Mean Absolute Error (MAE): 0.008942
Median Absolute Deviation (MAD): 0.006004
R-squared: 0.444024
Directional Accuracy: 0.738053


-----
## Prediction on Out-of-Sample Data ##

In [39]:
# Concatenate X_train and X_val
X_train_new = np.concatenate((X_train, X_val), axis=0)

# If you also want to combine the corresponding y values:
y_train_new = np.concatenate((y_train, y_val), axis=0)

In [40]:
print("X_train_new shape:", X_train_new.shape)
print("y_train_new shape:", y_train_new.shape)

print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train_new shape: (3207, 5)
y_train_new shape: (3207,)
X_test shape: (567, 5)
y_test shape: (567,)


In [41]:
# Apply Scaling
scaler_X = StandardScaler()

# Fit only on training data
X_train_new_scaled = scaler_X.fit_transform(X_train_new)

# Transform validation and test sets
X_test_scaled = scaler_X.transform(X_test)

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [42]:
lasso = Lasso()
# Set up a range of possible lambda values
param_grid = {
    'alpha' : [0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
}

kf = KFold(n_splits=10, shuffle=True, random_state=42)

lasso_cv = GridSearchCV(lasso, param_grid, cv=kf, n_jobs=-1)

lasso_cv.fit(X_train_new_scaled, y_train_new)

GridSearchCV(cv=KFold(n_splits=10, random_state=42, shuffle=True),
             estimator=Lasso(), n_jobs=-1,
             param_grid={'alpha': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1.0, 10.0,
                                   100.0]})

In [43]:
# Get the best Lasso model
lasso_optimal = lasso_cv.best_estimator_

# General Summary
info = lasso_model_info(lasso_optimal, X_train_new_scaled, y_train_new)
for k, v in info.items():
    if isinstance(v, str):
        print(k)
    else:
        print(f"{k}: {v:.4f}")

# Get the best model from GridSearchCV
lasso_optimal = lasso_cv.best_estimator_

# Generate the detailed coefficient summary
summary_df = lasso_summary(
    lasso_optimal,
    X_train_scaled,
    y_train,
    feature_names=X_train.columns.tolist()
)

# Display the summary
print(summary_df)

============GENERAL SUMMARY OF MODEL ============
R-squared: 0.5292
Adjusted R-squared: 0.5285
Log-Likelihood: 9607.4217
AIC: -19204.8434
BIC: -19174.4780
Durbin-Watson: 1.9996
Skew: -0.0730
Kurtosis: 10.8977
Jarque-Bera: 8337.4763
Prob(JB): 0.0000
Condition Number: 1.8044
============END OF GENERAL SUMMARY ============
                       coef   std err          t     P>|t|
const              0.001103  0.000231   4.777126  0.000002
HIST_CALL_IMP_VOL -0.000936  0.000233  -4.010198  0.000062
Mkt-RF             0.012890  0.000248  52.012555  0.000000
SMB               -0.000758  0.000262  -2.895161  0.003821
HML               -0.003848  0.000251 -15.337397  0.000000
RMW                0.002831  0.000264  10.729827  0.000000


In [44]:
# Optimal lambda automatically selected and stored in 'ridge_cv_model.alpha_'
y_pred = lasso_cv.predict(X_test_scaled)

In [45]:
# Evaluation metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
mad = median_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
direction_acc = directional_accuracy(y_test, y_pred)

# Print evaluation results
print(f"Mean Squared Error (MSE): {mse:.6f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.6f}")
print(f"Mean Absolute Error (MAE): {mae:.6f}")
print(f"Median Absolute Deviation (MAD): {mad:.6f}")
print(f"R-squared: {r2:.6f}")
print(f"Directional Accuracy: {direction_acc:.6f}")

Mean Squared Error (MSE): 0.000138
Root Mean Squared Error (RMSE): 0.011753
Mean Absolute Error (MAE): 0.008355
Median Absolute Deviation (MAD): 0.005973
R-squared: 0.529218
Directional Accuracy: 0.740283
